# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1:Alejandro Andres Velasco Santinato
*   Alumno 2:Daniela Leal Rivero
*   Alumno 3:John Briceño Diaz






---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```


---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [9]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
import os
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [ ]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

---
### 1.4. Instalar librerías necesarias

NOTA: esto es un fix para colab hay que ejecutarlo y cuando pida reiniciar el entorno se reinicia, despues de reiniciarlo simplemente ejecuten lo demas y deje esto como esta

In [ ]:

if IN_COLAB:
  !pip uninstall tensorflow -y
  !pip install tensorflow==2.15

In [4]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5

else:
  %pip install numpy==1.23.5
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow==2.10.1
  %pip install torch==2.0.1
  %pip install agents==1.4.0

     ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
      --------------------------------------- 0.0/1.6 MB 660.6 kB/s eta 0:00:03
     - -------------------------------------- 0.1/1.6 MB 656.4 kB/s eta 0:00:03
     -- ------------------------------------- 0.1/1.6 MB 939.4 kB/s eta 0:00:02
     --- ------------------------------------ 0.2/1.6 MB 833.5 kB/s eta 0:00:02
     ----- ---------------------------------- 0.2/1.6 MB 888.4 kB/s eta 0:00:02
     ------ --------------------------------- 0.3/1.6 MB 927.4 kB/s eta 0:00:02
     ------- -------------------------------- 0.3/1.6 MB 999.9 kB/s eta 0:00:02
     -------- ------------------------------- 0.3/1.6 MB 952.6 kB/s eta 0:00:02
     --------- ------------------------------ 0.4/1.6 MB 933.2 kB/s eta 0:00:02
     ---------- ----------------------------- 0.4/1.6 MB 904.2 kB/s eta 0:00:02
     ----------- ---------------------------- 0.5/1.6 MB 909.8 kB/s eta 0:00:02
     ------------ --------------------------- 0.5

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Cloning https://github.com/Kojoley/atari-py.git to c:\users\john-\appdata\local\temp\pip-req-build-jrqu27d6
  Resolved https://github.com/Kojoley/atari-py.git to commit 86a1e05c0a95e9e6233c3a413521fdb34ca8a089
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for atari-py: filename=atari_py-1.2.2-cp310-cp310-win_amd64.whl size=551738 sha256=bb5e999327982040da54ce6b60614acb89fa5e354725dbf77ab2ef2df97e64d4
  Stored in directory: C:\Users\john-\AppData\Local\Temp\pip-ephem-wheel-cache-rrk3fuhy\wheels\6c\9b\f1\8a80a63a233d6f4eb2e2ee8c7357f4875f21c3ffc7f2613f9f
Successfully built atari-py
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/Kojoley/atari-py.git 'C:\Users\john-\AppData\Local\Temp\pip-req-build-jrqu27d6'

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/172.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/172.3 MB ? eta -:--:--
     -------------------------------------- 0.0/172.3 MB 653.6 kB/s eta 0:04:24
     ---------------------------------------- 0.1/172.3 MB 1.0 MB/s eta 0:02:47
     ---------------------------------------- 0.2/172.3 MB 1.0 MB/s eta 0:02:50
     -------------------------------------- 0.2/172.3 MB 908.0 kB/s eta 0:03:10
     -------------------------------------- 0.2/172.3 MB 860.2 kB/s eta 0:03:21
     -------------------------------------- 0.3/172.3 MB 827.5 kB/s eta 0:03:28
     -------------------------------------- 0.3/172.3 MB 842.9 kB/s eta 0:03:25
     -------------------------------------- 0.3/172.3 MB 785.7 kB/s eta 0:03:39
     -------------------------------------- 0.4/172.3 MB 794.9 kB/s eta 0:03:37
     -------------------------------------- 0.4/172.3 MB 804.0 kB/s eta 0:03:34
     -------------------------------------- 0.4/172.3 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/118.6 kB ? eta -:--:--
     ---------- ---------------------------- 30.7/118.6 kB 1.4 MB/s eta 0:00:01
     -------------------------------------- 118.6/118.6 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/118.1 kB ? eta -:--:--
     ------------- ------------------------- 41.0/118.1 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 118.1/118.1 kB 1.7 MB/s eta 0:00:00
  Created wheel for agents: filename=agents-1.4.0-py3-none-any.whl size=62722 sha256=5a3352d01dc10ae4e9edd353006d352a3250dae3b087ed46d5cc8dfa72978cd2
  Stored in directory: c:\users\john-\appdata\local\pip\cache\wheels\46\90\01\aa4bfe3189f87214f4d5825e677b42f6b3fb44125af0b175b4
Successfully built agents



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [2]:
from PIL import Image
import numpy as np
import gym
import keras
print(keras.__version__)
import tensorflow as tf

tf.keras.__version__ = keras.__version__
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

2.10.0


In [3]:
#Comprobando GPU
print(tf.config.experimental.list_physical_devices("GPU"))
print("Versión de TensorFlow:", tf.__version__)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Versión de TensorFlow: 2.10.1


#### Configuración base

In [4]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [5]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

1. Implementación de la red neuronal

In [6]:
def build_model(input_shape, nb_actions):
    model = Sequential()
    model.add(Permute((2, 3, 1), input_shape=input_shape))
    model.add(Convolution2D(32, (8, 8), strides=(4, 4), activation='relu'))
    model.add(Convolution2D(64, (4, 4), strides=(2, 2), activation='relu'))
    model.add(Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(nb_actions, activation='linear'))
    return model

input_shape = (WINDOW_LENGTH, INPUT_SHAPE[0], INPUT_SHAPE[1])
model = build_model(input_shape, nb_actions)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 permute (Permute)           (None, 84, 84, 4)         0         
                                                                 
 conv2d (Conv2D)             (None, 20, 20, 32)        8224      
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 64)          32832     
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 512)               1606144   
                                                                 
 dense_1 (Dense)             (None, 6)                 3

2. Implementación de la solución DQN

In [7]:
from tensorflow.keras.optimizers.legacy import Adam
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)

# Creando la policy
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=0.05, nb_steps=1000000)

# Creando el agente de DQN
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=AtariProcessor(), nb_steps_warmup=50000, gamma=.99, target_model_update=10000,
               train_interval=4, delta_clip=1.)

# Compilando el agente
dqn.compile(Adam(learning_rate=0.00025), metrics=['mae'])

# Logica para almacenar los pesos del modelo
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=50000)]
callbacks += [FileLogger('dqn_{}_log.json'.format(env_name), interval=100)]


In [11]:
# --- Parte de Entrenamiento con Checkpoints en Google Drive ---

# Define el directorio para los checkpoints dentro de tu carpeta de Google Drive
checkpoint_dir = os.path.join(drive_root, 'checkpoints')
# Crea el directorio si no existe (exist_ok=True evita errores si ya existe)
os.makedirs(checkpoint_dir, exist_ok=True)

# Define el nombre de archivo para guardar los pesos, incluyendo la ruta a tu carpeta de Drive
# {step} será reemplazado automáticamente por el número de pasos realizados durante el entrenamiento
checkpoint_weights_filename = os.path.join(checkpoint_dir, 'dqn_' + env_name + '_weights_{step}.h5f')

# Define los callbacks para guardar los pesos y registrar el progreso
# ModelIntervalCheckpoint guarda los pesos en intervalos especificados
# FileLogger registra el progreso del entrenamiento en un archivo JSON
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=50000)]
callbacks += [FileLogger(os.path.join(drive_root, 'dqn_{}_log.json'.format(env_name)), interval=100)]


# --- Iniciar Entrenamiento ---
print("\nIniciando el entrenamiento de DQN...")
# Puedes ajustar 'nb_steps' según tus recursos computacionales y la duración deseada del entrenamiento.
# Para un buen rendimiento en Atari, esto a menudo requiere millones de pasos.
# Comienza con un número menor como 500,000 para ver el progreso, luego auméntalo.
final_weights_filename = os.path.join(drive_root, 'dqn_{}_final_weights.h5f'.format(env_name))
#dqn.load_weights(final_weights_filename) # Carga los pesos finales guardados anteriormente
dqn.fit(env, callbacks=callbacks, nb_steps=2000000, log_interval=10000, visualize=False, verbose=2)

# Después de entrenar, guarda los pesos finales
final_weights_filename = os.path.join(drive_root, 'dqn_{}_final_weights.h5f'.format(env_name))
#cargar pesos anteriores
dqn.save_weights(final_weights_filename, overwrite=True)
print(f"\nEntrenamiento finalizado. Pesos finales guardados en: {final_weights_filename}")

# --- Parte de prueba para calcular la recompensa media (después del entrenamiento) ---
print("\nIniciando la fase de prueba...")
# Carga los pesos finales para la prueba (o un checkpoint específico si quieres probar un modelo intermedio)
dqn.load_weights(final_weights_filename) # Carga los pesos finales guardados anteriormente
dqn.test(env, nb_episodes=10, visualize=False)
print("Prueba completada.")


Iniciando el entrenamiento de DQN...
Training for 2000000 steps ...


c:\Users\john-\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


     420/2000000: episode: 1, duration: 8.964s, episode steps: 420, steps per second:  47, episode reward:  6.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.479 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    1131/2000000: episode: 2, duration: 3.979s, episode steps: 711, steps per second: 179, episode reward:  7.000, mean reward:  0.010 [ 0.000,  1.000], mean action: 2.419 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    1941/2000000: episode: 3, duration: 4.310s, episode steps: 810, steps per second: 188, episode reward: 10.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 2.410 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    2831/2000000: episode: 4, duration: 4.779s, episode steps: 890, steps per second: 186, episode reward: 12.000, mean reward:  0.013 [ 0.000,  1.000], mean action: 2.392 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
    3206/2000000: episode: 5, duration: 2.117s, episode steps: 375, 

c:\Users\john-\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   50534/2000000: episode: 74, duration: 22.686s, episode steps: 770, steps per second:  34, episode reward: 12.000, mean reward:  0.016 [ 0.000,  1.000], mean action: 2.530 [0.000, 5.000],  loss: 0.005935, mae: 0.026032, mean_q: 0.044395, mean_eps: 0.954759
   51005/2000000: episode: 75, duration: 16.986s, episode steps: 471, steps per second:  28, episode reward:  5.000, mean reward:  0.011 [ 0.000,  1.000], mean action: 2.423 [0.000, 5.000],  loss: 0.005904, mae: 0.026882, mean_q: 0.041804, mean_eps: 0.954307
   51721/2000000: episode: 76, duration: 25.653s, episode steps: 716, steps per second:  28, episode reward: 12.000, mean reward:  0.017 [ 0.000,  1.000], mean action: 2.451 [0.000, 5.000],  loss: 0.006332, mae: 0.029129, mean_q: 0.040920, mean_eps: 0.953772
   52422/2000000: episode: 77, duration: 24.790s, episode steps: 701, steps per second:  28, episode reward:  7.000, mean reward:  0.010 [ 0.000,  1.000], mean action: 2.536 [0.000, 5.000],  loss: 0.005758, mae: 0.028427, m

In [ ]:
# Testing part to calculate the mean reward
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

3. Justificación de los parámetros seleccionados y de los resultados obtenidos

---